%Programs step1, 2, an 3 are for a reversed refraction profile
%to obtain the input for seisdip.exe a DOS dipping layer refraction program.
%step1 plots the forward and reverse seismic sections. step 2 
%is used to pick first arrivals.  Step3 allows you to time the branches and
%obtain velocities and intercepts needed for seisdip. You will need to set nb in step 3 
%which determiones the number of branches to be picked in the data.
%In step 3 you can ensure reciprocity (necessary to < 10 ms if seisdip is to run
% successfully) by picking the total travel times of the forward branches as one 
%of the 2 points defining a reverse branch.  
%Make sure if you change files to change variable names
%seis=, in the code; 
%nodata=no. data points; gain is amplification; nostation is number t
%Clear previous data and plots.
%INPUT ascii files with seismgrams in columns (e.g., fl103.txt, fl106.txt)
%      from RAS saved as ascii files.
%      nodata=number of data
%      gain=amplification
%      nostations=number of seismograms to be used
%      station1= first station to be used
%      =time interval between data points
%      dx=spacing between stations
%Associated Programs plotseis.m

% CHANGE IN 4 PLACES 
% LINES 31-32 INPUT shots
% LINES 38-42 input data
% LINES 54, 66  Shot number from lines 31,32
% LINES 62, 69  Shot number from lines 31,32

In [1]:
import pandas as pd, numpy as np, matplotlib.pyplot as plt
from obspy.signal.filter import bandpass
from obspy.signal.detrend import polynomial
%matplotlib tk

In [26]:
nodata=300
dt=1/1000
nostations=48
dx=5
rlength=(nostations)*dx
station1=0
stationn=station1+nostations-1
numpicks=49
nbranch=2
f0=40;width=f0^2
fflag=1

seisS1=np.genfromtxt('data/Saturday/South/04.txt') #direct wave
seisS2=np.genfromtxt('data/Saturday/South/07.txt') #direct wave
tseis1='South'
seis1=np.hstack((seisS1,seisS2))
seisN1=np.genfromtxt('data/Saturday/North/01.txt'); #reverse wave
seisN2=np.genfromtxt('data/Saturday/North/05.txt'); #reverse wave

tseis2='North'
seis2=np.hstack((seisN1,seisN2))



In [38]:
def plotseis(seis):
    fig =plt.figure(figsize=(18,10))
    a = fig.add_subplot(111)
    for i in range(nostations):
    #     tr.normalize()
        tr=seis[:nodata,i]
    #     tr = minmax_scale(tr, feature_range=(-1,1))
        tr=polynomial(tr, order=3, plot=False)  
        tr=bandpass(tr,0.1,30,1/dt)
        tr=tr/tr.max()*5
        dist = dx*i+dx
        y = np.arange(nodata)*dt
        x = tr+dist
        a.plot(x,y,'k')
        a.fill_betweenx(y,x,dist,x > dist, color='r', alpha = 0.8)
    a.set_xlabel("Station Dist")    

    a.set_ylabel("sec")    
    a.set_ylim(0,0.3)
    a.set_xlim(0,250)

    plt.tight_layout()
    plt.gca().invert_yaxis()

#     plt.show()
    return fig,a
    
def onclick(event):
    ix,iy=event.xdata, event.ydata    
    plt.plot(ix,iy,'b+')
    plt.show()
    print(ix,iy)
    if len(picks)<nostations:
        picks.append((ix,iy))
    else:
        fig.canvas.mpl_disconnect(cid)

    
    


In [36]:
picks=[]
fig, a =plotseis(seis1)
cid = fig.canvas.mpl_connect('button_press_event',onclick)

4.4474598243 0.018725123628
9.62784327063 0.0294717163189
13.8940414029 0.0385900373899
19.6838817253 0.0483596671089
25.1689936096 0.0597575684477
29.7399201799 0.0639910746593
35.0726678453 0.0721324327584
39.6435944155 0.0770172476179
44.3668852048 0.0819020624774
49.8519970892 0.0916716921964
55.1847447545 0.0972078157038
59.4509428868 0.102418284887
64.9360547712 0.110559642986
69.5069813414 0.112839223254
75.2968216638 0.118701001086
79.4106555771 0.124888433241
83.5244894903 0.128144976481
92.0568857549 0.134332408636
94.9518059161 0.138240260523
99.3703682674 0.139868532143
103.484202181 0.146707272947
108.20749297 0.15028947051
114.759154387 0.155499939694
120.091902053 0.161687371849
124.510464404 0.165920878061
129.081390974 0.17080569292
134.566502859 0.175039199132
138.832700991 0.177318779399
144.317812875 0.180249668315
150.107653198 0.182529248583
155.89749352 0.187739717766
159.554234776 0.19132191533
165.039346661 0.195881075865
169.153180574 0.199137619105
175.095385

In [37]:
len(picks)

49